error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [ ]:
# DAY 15 (10th May)
import cv2
vid = cv2.VideoCapture(0)
fd=cv2.CascadeClassifier(
            cv2.data.haarcascades+'haarcascade_frontalface_default.xml'
)

frameCount=0
name = input('Enter your name:')
while True:
    flag, img=vid.read()
    if flag:
        faces = fd.detectMultiScale(
            cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),
            scaleFactor = 1.1,
            minNeighbors=5,
            minSize = (50,50)
        )

        if len(faces)==1:
            x,y,w,h = faces[0]
            img_face = img[y:y+h, x:x+w, :].copy()
            cv2.imwrite(f'{name}_{frameCount}.png', img_face)
            frameCount += 1 
        
        if frameCount == 20:
            break

    cv2.imshow('preview', img)
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    vid.release()
            

In [ ]:
# Day 16 (12th May)

In [ ]:
# FACE REGISTERING 
import cv2
import pandas as pd
import face_recognition as fr

vid = cv2.VideoCapture(0)
fd=cv2.CascadeClassifier(
            cv2.data.haarcascades+'haarcascade_frontalface_default.xml'
)
try:
    face_db = pd.read_csv('faces_data.tsv', index_col=0, sep='\t')
    data = {
        'name' : face_db['name'].values.tolist(),
        'encoding' : face_db['encoding'].values.tolist(),
    }
except Exception as e:
    print(e)
    data = {'name' : [], 'encoding' : []}

names=data['name']
enc=data['encoding']
frameCount=0
frameLimit=20

name = input('Enter your name:')
while True:
    flag, img=vid.read()
    if flag:
        faces = fd.detectMultiScale(
            cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),
            scaleFactor = 1.1,
            minNeighbors=5,
            minSize = (50,50)
        )

        if len(faces)==1:
            x,y,w,h = faces[0]
            img_face = img[y:y+h, x:x+w, :].copy()
            img_face= cv2.resize(img_face,(400,400),
                                 interpolation=cv2.INTER_CUBIC)
            #cv2.imwrite(f'{name}_{frameCount}.png', img_face)
            face_encoding = fr.face_encodings(img_face)
            if len(face_encoding) == 1:
                enc.append(face_encoding[0].tolist())
                names.append(name)
                frameCount += 1
                print(frameCount)
                cv2.putText(
                            img, str(frameCount),
                            (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5,
                            (0,0,255), 8
                        )
                if frameCount == frameLimit:
                    data = {'name':names, 'encoding':enc}
                    pd.DataFrame(data).to_csv('faces_data.tsv', sep='\t'             #why here .tsv is written
                                              )
                    break
        
        

    cv2.imshow('preview', img)
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
cv2.waitKey(1)
cv2.destroyAllWindows()
vid.release()
            

[Errno 2] No such file or directory: 'faces_data.tsv'
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
# Recognition Step
#######################

import cv2
import pandas as pd
import face_recognition as fr

import numpy as np

vid = cv2.VideoCapture(0)
fd=cv2.CascadeClassifier(
            cv2.data.haarcascades+'haarcascade_frontalface_default.xml'
)

try:
    face_db = pd.read_csv('faces_data.tsv', index_col=0, sep='\t')
    data = {
        'name' : face_db['name'].values.tolist(),
        'encoding' : face_db['encoding'].values.tolist(),
    }
except Exception as e:
    print(e)
    data = {'name' : [], 'encoding' : []}



while True:
    flag, img=vid.read()
    if flag:
        faces = fd.detectMultiScale(
            cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),
            scaleFactor = 1.1,
            minNeighbors=5,
            minSize = (50,50)
        )

        if len(faces)==1:
            x,y,w,h = faces[0]
            img_face = img[y:y+h, x:x+w, :].copy()
            img_face= cv2.resize(img_face,(400,400),
                                 interpolation=cv2.INTER_CUBIC)
            #cv2.imwrite(f'{name}_{frameCount}.png', img_face)
            face_encoding = fr.face_encodings(img_face)

            if len(face_encoding) == 1:
                for ind, enc_value in enumerate(data['encoding']):
                    matched = fr.compare_faces(
                        face_encoding, np.array(eval(enc_value))
                    )[0]
                    if matched == True:
                        cv2.putText(
                            img, data['name'] [ind],
                            (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5,
                            (0,0,255), 8
                        )
                        break

        cv2.imshow('preview', img)
        key=cv2.waitKey(1)
        if key == ord('q'):
            break
cv2.waitKey(1)
cv2.destroyAllWindows()
vid.release()
            

In [ ]:
import face_recognition